In [1]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'

spark_version = 'spark-3.2.3'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [3,127 kB]
Hit:13 https://developer.download.nvidia.com/c

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Hashing").getOrCreate()

In [3]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [ ]:
# Read in data
from pyspark import SparkFiles
df_88 = spark.read.csv("lyrics_88.csv", header=True)

# Show DataFrame
df_88.show()

+---+-----------+--------------------+--------------------+
|_c0|artist_name|              lyrics|    lyrics_tokenized|
+---+-----------+--------------------+--------------------+
|  0|       BIBI| watch me dance w...|['watch', 'me', '...|
|  1|       BIBI| dice and ice cub...|['dice', 'and', '...|
|  2|       BIBI| champagne it mig...|['champagne', 'it...|
|  3|       BIBI| bad sad and mad ...|['bad', 'sad', 'a...|
|  5|       BIBI|                    |                  []|
|  6|       BIBI| i like that i li...|['i', 'like', 'th...|
|  7|       BIBI| maybe if iwoke u...|['maybe', 'if', '...|
|  8|       BIBI| jotto mm mm jott...|['jotto', 'mm', '...|
| 10|       BIBI| pado if someone ...|['pado', 'if', 's...|
| 16|       BIBI| i need you when ...|['i', 'need', 'yo...|
| 18|       BIBI| feed me feed me ...|['feed', 'me', 'f...|
| 19|       BIBI| i need you got m...|['i', 'need', 'yo...|
| 20|       BIBI| fill me fill you...|['fill', 'me', 'f...|
| 21|       BIBI|nobody told me i ...|['

In [5]:
# Read in data
from pyspark import SparkFiles
df_88_line = spark.read.csv("lyrics88_by_line.csv", header=True)

# Show DataFrame
df_88_line.show()

+---+------+--------------------+
|_c0|artist|    formatted_lyrics|
+---+------+--------------------+
|  0|     0|      Watch me dance|
|  1|     0|      Watch me sing |
|  2|     0|      Watch me dance|
|  3|     0|      Watch me sing |
|  4|     0|I got mercy, I go...|
|  5|     0|Desperado with th...|
|  6|     0|Throw a rose on y...|
|  7|     0|Mask of Zorro I'm...|
|  8|     0|        Look scared |
|  9|     0|  Dice and Ice cube |
| 10|     0| Home girls bitches |
| 11|     0|Spin it, spin it,...|
| 12|     0|Chips andchips an...|
| 13|     0|Demon under the t...|
| 14|     0|Risk it, risk it,...|
| 15|     0|    Yeah, yeah, yeah|
| 16|     0|Risk it, risk it,...|
| 17|     0|B-b-bitches, Ice ...|
| 18|     0|Bitches, Ice and ...|
| 19|     0|Bitches, Ice and ...|
+---+------+--------------------+
only showing top 20 rows



In [ ]:
newdf_88= df_88.drop("lyrics_tokenized")
newdf_88.show()

+---+-----------+--------------------+
|_c0|artist_name|              lyrics|
+---+-----------+--------------------+
|  0|       BIBI| watch me dance w...|
|  1|       BIBI| dice and ice cub...|
|  2|       BIBI| champagne it mig...|
|  3|       BIBI| bad sad and mad ...|
|  5|       BIBI|                    |
|  6|       BIBI| i like that i li...|
|  7|       BIBI| maybe if iwoke u...|
|  8|       BIBI| jotto mm mm jott...|
| 10|       BIBI| pado if someone ...|
| 16|       BIBI| i need you when ...|
| 18|       BIBI| feed me feed me ...|
| 19|       BIBI| i need you got m...|
| 20|       BIBI| fill me fill you...|
| 21|       BIBI|nobody told me i ...|
| 22|       BIBI| wet nightmare po...|
| 25|       BIBI|open your eyes mi...|
| 29|       BIBI| fly with me with...|
| 30|       BIBI| timeless inside ...|
| 31|       BIBI| give more care l...|
| 32|   CHUNG HA| sparkling come a...|
+---+-----------+--------------------+
only showing top 20 rows



In [6]:
newdf_line_88= df_88_line.select("artist", "formatted_lyrics")
newdf_line_88.show()

+------+--------------------+
|artist|    formatted_lyrics|
+------+--------------------+
|     0|      Watch me dance|
|     0|      Watch me sing |
|     0|      Watch me dance|
|     0|      Watch me sing |
|     0|I got mercy, I go...|
|     0|Desperado with th...|
|     0|Throw a rose on y...|
|     0|Mask of Zorro I'm...|
|     0|        Look scared |
|     0|  Dice and Ice cube |
|     0| Home girls bitches |
|     0|Spin it, spin it,...|
|     0|Chips andchips an...|
|     0|Demon under the t...|
|     0|Risk it, risk it,...|
|     0|    Yeah, yeah, yeah|
|     0|Risk it, risk it,...|
|     0|B-b-bitches, Ice ...|
|     0|Bitches, Ice and ...|
|     0|Bitches, Ice and ...|
+------+--------------------+
only showing top 20 rows



In [13]:
# Tokenize DataFrame
tokened = Tokenizer(inputCol="lyrics", outputCol="lyrics_words")

In [7]:
tokened_line = Tokenizer(inputCol = "formatted_lyrics", outputCol = "lyrics_words")

In [ ]:
# Transform DataFrame
tokenized = tokened.transform(newdf_88)
tokenized.show()

+---+-----------+--------------------+--------------------+
|_c0|artist_name|              lyrics|        lyrics_words|
+---+-----------+--------------------+--------------------+
|  0|       BIBI| watch me dance w...|[, watch, me, dan...|
|  1|       BIBI| dice and ice cub...|[, dice, and, ice...|
|  2|       BIBI| champagne it mig...|[, champagne, it,...|
|  3|       BIBI| bad sad and mad ...|[, bad, sad, and,...|
|  5|       BIBI|                    |                  []|
|  6|       BIBI| i like that i li...|[, i, like, that,...|
|  7|       BIBI| maybe if iwoke u...|[, maybe, if, iwo...|
|  8|       BIBI| jotto mm mm jott...|[, jotto, mm, mm,...|
| 10|       BIBI| pado if someone ...|[, pado, if, some...|
| 16|       BIBI| i need you when ...|[, i, need, you, ...|
| 18|       BIBI| feed me feed me ...|[, feed, me, feed...|
| 19|       BIBI| i need you got m...|[, i, need, you, ...|
| 20|       BIBI| fill me fill you...|[, fill, me, fill...|
| 21|       BIBI|nobody told me i ...|[n

In [8]:
tokenized_line = tokened_line.transform(newdf_line_88)
tokenized_line.show()

+------+--------------------+--------------------+
|artist|    formatted_lyrics|        lyrics_words|
+------+--------------------+--------------------+
|     0|      Watch me dance|  [watch, me, dance]|
|     0|      Watch me sing |   [watch, me, sing]|
|     0|      Watch me dance|  [watch, me, dance]|
|     0|      Watch me sing |   [watch, me, sing]|
|     0|I got mercy, I go...|[i, got, mercy,, ...|
|     0|Desperado with th...|[desperado, with,...|
|     0|Throw a rose on y...|[throw, a, rose, ...|
|     0|Mask of Zorro I'm...|[mask, of, zorro,...|
|     0|        Look scared |      [look, scared]|
|     0|  Dice and Ice cube |[dice, and, ice, ...|
|     0| Home girls bitches |[home, girls, bit...|
|     0|Spin it, spin it,...|[spin, it,, spin,...|
|     0|Chips andchips an...|[chips, andchips,...|
|     0|Demon under the t...|[demon, under, th...|
|     0|Risk it, risk it,...|[risk, it,, risk,...|
|     0|    Yeah, yeah, yeah|[yeah,, yeah,, yeah]|
|     0|Risk it, risk it,...|[r

In [9]:
spark = SparkSession.builder.appName("StopWords").getOrCreate()
from pyspark.ml.feature import StopWordsRemover

In [10]:
# Remove Stop words
remover = StopWordsRemover(inputCol="lyrics_words", outputCol="filtered_lyrics")

In [ ]:
no_stop_words_df = remover.transform(tokenized)
no_stop_words_df.show()

+---+-----------+--------------------+--------------------+--------------------+
|_c0|artist_name|              lyrics|        lyrics_words|     filtered_lyrics|
+---+-----------+--------------------+--------------------+--------------------+
|  0|       BIBI| watch me dance w...|[, watch, me, dan...|[, watch, dance, ...|
|  1|       BIBI| dice and ice cub...|[, dice, and, ice...|[, dice, ice, cub...|
|  2|       BIBI| champagne it mig...|[, champagne, it,...|[, champagne, mig...|
|  3|       BIBI| bad sad and mad ...|[, bad, sad, and,...|[, bad, sad, mad,...|
|  5|       BIBI|                    |                  []|                  []|
|  6|       BIBI| i like that i li...|[, i, like, that,...|[, like, like, le...|
|  7|       BIBI| maybe if iwoke u...|[, maybe, if, iwo...|[, maybe, iwoke, ...|
|  8|       BIBI| jotto mm mm jott...|[, jotto, mm, mm,...|[, jotto, mm, mm,...|
| 10|       BIBI| pado if someone ...|[, pado, if, some...|[, pado, someone,...|
| 16|       BIBI| i need you

In [11]:
no_stop_words_df_line = remover.transform(tokenized_line)
no_stop_words_df_line.show()

+------+--------------------+--------------------+--------------------+
|artist|    formatted_lyrics|        lyrics_words|     filtered_lyrics|
+------+--------------------+--------------------+--------------------+
|     0|      Watch me dance|  [watch, me, dance]|      [watch, dance]|
|     0|      Watch me sing |   [watch, me, sing]|       [watch, sing]|
|     0|      Watch me dance|  [watch, me, dance]|      [watch, dance]|
|     0|      Watch me sing |   [watch, me, sing]|       [watch, sing]|
|     0|I got mercy, I go...|[i, got, mercy,, ...|[got, mercy,, got...|
|     0|Desperado with th...|[desperado, with,...| [desperado, scales]|
|     0|Throw a rose on y...|[throw, a, rose, ...| [throw, rose, body]|
|     0|Mask of Zorro I'm...|[mask, of, zorro,...|[mask, zorro, wan...|
|     0|        Look scared |      [look, scared]|      [look, scared]|
|     0|  Dice and Ice cube |[dice, and, ice, ...|   [dice, ice, cube]|
|     0| Home girls bitches |[home, girls, bit...|[home, girls, 

In [12]:
no_stop_words_df_line.select("filtered_lyrics").show(truncate=False)

+-------------------------------------------------------+
|filtered_lyrics                                        |
+-------------------------------------------------------+
|[watch, dance]                                         |
|[watch, sing]                                          |
|[watch, dance]                                         |
|[watch, sing]                                          |
|[got, mercy,, got, grace]                              |
|[desperado, scales]                                    |
|[throw, rose, body]                                    |
|[mask, zorro, wanted]                                  |
|[look, scared]                                         |
|[dice, ice, cube]                                      |
|[home, girls, bitches]                                 |
|[spin, it,, spin, it,pussy]                            |
|[chips, andchips, chips]                               |
|[demon, table]                                         |
|[risk, it,, r

In [14]:
no_stop_words_df_line.select("filtered_lyrics").show(truncate=False)

+-------------------------------------------------------+
|filtered_lyrics                                        |
+-------------------------------------------------------+
|[watch, dance]                                         |
|[watch, sing]                                          |
|[watch, dance]                                         |
|[watch, sing]                                          |
|[got, mercy,, got, grace]                              |
|[desperado, scales]                                    |
|[throw, rose, body]                                    |
|[mask, zorro, wanted]                                  |
|[look, scared]                                         |
|[dice, ice, cube]                                      |
|[home, girls, bitches]                                 |
|[spin, it,, spin, it,pussy]                            |
|[chips, andchips, chips]                               |
|[demon, table]                                         |
|[risk, it,, r

In [15]:
from pyspark.ml.feature import HashingTF, IDF

In [16]:
hashing = HashingTF(inputCol="filtered_lyrics", outputCol="hashedValues")

In [18]:
hashed_df = hashing.transform(no_stop_words_df_line)
hashed_df.show(truncate=False)

+------+-----------------------------------------------------+----------------------------------------------------------------+-------------------------------------------------------+--------------------------------------------------------------------------------------+
|artist|formatted_lyrics                                     |lyrics_words                                                    |filtered_lyrics                                        |hashedValues                                                                          |
+------+-----------------------------------------------------+----------------------------------------------------------------+-------------------------------------------------------+--------------------------------------------------------------------------------------+
|0     |Watch me dance                                       |[watch, me, dance]                                              |[watch, dance]                                         |(262

In [23]:
hashed_df = hashing.transform(no_stop_words_df_line)
hashed_df.show(truncate=False)

+------+-----------------------------------------------------+----------------------------------------------------------------+-------------------------------------------------------+--------------------------------------------------------------------------------------+
|artist|formatted_lyrics                                     |lyrics_words                                                    |filtered_lyrics                                        |hashedValues                                                                          |
+------+-----------------------------------------------------+----------------------------------------------------------------+-------------------------------------------------------+--------------------------------------------------------------------------------------+
|0     |Watch me dance                                       |[watch, me, dance]                                              |[watch, dance]                                         |(262

In [24]:
# Fit the IDF on the data set 
idf = IDF(inputCol="hashedValues", outputCol="features")
idfModel = idf.fit(hashed_df)
rescaledData = idfModel.transform(hashed_df)

In [25]:
rescaledData.select("lyrics_words", "features").show(truncate=False)

+----------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|lyrics_words                                                    |features                                                                                                                                                                                   |
+----------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[watch, me, dance]                                              |(262144,[33075,229604],[5.782084537851953,5.973139774614663])                                                                                                            

In [19]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='artist_name',outputCol='label')
tokenizer = Tokenizer(inputCol="lyrics", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [20]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='artist',outputCol='label')
tokenizer = Tokenizer(inputCol="formatted_lyrics", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [21]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token'], outputCol='features')

In [22]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[pos_neg_to_num, tokenizer, stopremove, hashingTF, idf, clean_up])

In [ ]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(newdf_88)
cleaned = cleaner.transform(newdf_88)

In [23]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(newdf_line_88)
cleaned = cleaner.transform(newdf_line_88)

In [24]:
cleaned.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
| 12.0|(262144,[33075,22...|
| 12.0|(262144,[117155,2...|
| 12.0|(262144,[33075,22...|
| 12.0|(262144,[117155,2...|
| 12.0|(262144,[34919,47...|
| 12.0|(262144,[141796,2...|
| 12.0|(262144,[34121,48...|
| 12.0|(262144,[97006,18...|
| 12.0|(262144,[165682,2...|
| 12.0|(262144,[84809,12...|
| 12.0|(262144,[17893,22...|
| 12.0|(262144,[46029,11...|
| 12.0|(262144,[194321,2...|
| 12.0|(262144,[114395,2...|
| 12.0|(262144,[5381,460...|
| 12.0|(262144,[82065,93...|
| 12.0|(262144,[5381,460...|
| 12.0|(262144,[13781,17...|
| 12.0|(262144,[13781,17...|
| 12.0|(262144,[13781,17...|
+-----+--------------------+
only showing top 20 rows



In [25]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [26]:
# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show(5)

+------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|artist|    formatted_lyrics|label|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|     0| got my weapon no...| 12.0|[, got, my, weapo...|[, got, weapon, g...|(262144,[81672,13...|(262144,[81672,13...|(262144,[81672,13...|[-776.20657819002...|[1.0,4.6524076649...|       0.0|
|     0|     pills and wine | 12.0|[, pills, and, wine]|     [, pills, wine]|(262144,[203513,2...|(262144,[203513,2...|(262144,[203513,2...|[-197.30021326057...|[3.23662785444732...|       7.0|
|     0|"Least loved will...| 

In [27]:
cleaned.select("artist", "label").distinct().show()

+------+-----+
|artist|label|
+------+-----+
|     4|  6.0|
|     8|  7.0|
|     9| 11.0|
|     7|  5.0|
|     2|  2.0|
|    11| 13.0|
|    12|  9.0|
|    10|  8.0|
|    13|  1.0|
|     0| 12.0|
|     6|  3.0|
|     5| 10.0|
|     3|  0.0|
|     1|  4.0|
+------+-----+



In [28]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting artists was: %f" % acc)

Accuracy of model at predicting artists was: 0.464127


In [42]:
# Test model on generated lyrics
rb_hypetime = spark.read.text('RichBrian-HypeTime.txt')
rb_hypetime.show(truncate=False)


+-------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                      |
+-------------------------------------------------------------------------------------------------------------------------------------------+
|Artist: Rich Brian                                                                                                                         |
|                                                                                                                                           |
|Topic: Hype Time                                                                                                                           |
|                                                                                                                                           |
|Lyric

In [43]:
rb_hypetime = rb_hypetime.where(rb_hypetime.value.isin(['Artist: Rich Brian ','Topic: Hype Time ', 'Lyrics:', '[Chorus]', '', ' '])==False)
rb_hypetime.show()

+--------------------+
|               value|
+--------------------+
|If I want the hyp...|
|If I want the mon...|
|If I want a bad b...|
|I don't need your...|
|I was born in ' ,...|
|You don't know me...|
|I'm so focused on...|
|When you ready fo...|
|When you ready to...|
|When you ready fo...|
|Fuck what these n...|
+--------------------+



In [44]:
from pyspark.sql.functions import lit


In [45]:
# Add artist_name and lyrics columns
rb_hypetime = rb_hypetime.withColumn("artist", lit('Rich Brian')).select("artist", "value")
rb_hypetime.show()

+----------+--------------------+
|    artist|               value|
+----------+--------------------+
|Rich Brian|If I want the hyp...|
|Rich Brian|If I want the mon...|
|Rich Brian|If I want a bad b...|
|Rich Brian|I don't need your...|
|Rich Brian|I was born in ' ,...|
|Rich Brian|You don't know me...|
|Rich Brian|I'm so focused on...|
|Rich Brian|When you ready fo...|
|Rich Brian|When you ready to...|
|Rich Brian|When you ready fo...|
|Rich Brian|Fuck what these n...|
+----------+--------------------+



In [46]:
rb_hypetime = rb_hypetime.withColumnRenamed("value","formatted_lyrics")
rb_hypetime.show()

+----------+--------------------+
|    artist|    formatted_lyrics|
+----------+--------------------+
|Rich Brian|If I want the hyp...|
|Rich Brian|If I want the mon...|
|Rich Brian|If I want a bad b...|
|Rich Brian|I don't need your...|
|Rich Brian|I was born in ' ,...|
|Rich Brian|You don't know me...|
|Rich Brian|I'm so focused on...|
|Rich Brian|When you ready fo...|
|Rich Brian|When you ready to...|
|Rich Brian|When you ready fo...|
|Rich Brian|Fuck what these n...|
+----------+--------------------+



In [48]:
import pandas as pd

In [58]:
# Save lyrics as csv
rb_hypetime.toPandas().to_csv("/content/sample_data/rb_hypetime.csv")

In [37]:
rb_hypetime = rb_hypetime.withColumn("label", lit(0.0))
rb_hypetime.show()

+----------+--------------------+-----+
|    artist|    formatted_lyrics|label|
+----------+--------------------+-----+
|Rich Brian|If I want the hyp...|  0.0|
|Rich Brian|If I want the mon...|  0.0|
|Rich Brian|If I want a bad b...|  0.0|
|Rich Brian|I don't need your...|  0.0|
|Rich Brian|I was born in ' ,...|  0.0|
|Rich Brian|You don't know me...|  0.0|
|Rich Brian|I'm so focused on...|  0.0|
|Rich Brian|When you ready fo...|  0.0|
|Rich Brian|When you ready to...|  0.0|
|Rich Brian|When you ready fo...|  0.0|
|Rich Brian|Fuck what these n...|  0.0|
+----------+--------------------+-----+



In [38]:
artist_data_prep_pipeline = Pipeline(stages=[tokenizer, stopremove, hashingTF, idf, clean_up])

In [39]:
# Fit and transform the pipeline
rb_cleaner = artist_data_prep_pipeline.fit(rb_hypetime)
rb_cleaned = rb_cleaner.transform(rb_hypetime)

In [40]:
rb_test_results = predictor.transform(rb_cleaned)
rb_test_results.show()

+----------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    artist|    formatted_lyrics|label|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+----------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Rich Brian|If I want the hyp...|  0.0|[if, i, want, the...|[want, hype,, get...|(262144,[34994,63...|(262144,[34994,63...|(262144,[34994,63...|[-69.630388127142...|[0.48732310252362...|       1.0|
|Rich Brian|If I want the mon...|  0.0|[if, i, want, the...|[want, money,, we...|(262144,[28482,10...|(262144,[28482,10...|(262144,[28482,10...|[-104.92062106943...|[0.89270396870369...|       0.0|
|Rich Bria

In [41]:
rb_acc = acc_eval.evaluate(rb_test_results)
print("Accuracy of model at predicting artists was: %f" % rb_acc)

Accuracy of model at predicting artists was: 0.705882


In [63]:
# Test model on generated lyrics
joji_ye = spark.read.text('Joji-YoungEnergy.txt')

In [64]:
joji_ye = joji_ye.where(joji_ye.value.isin(['Artist: Joji ','Topic: Young Energy ', 'Lyrics:', '[Chorus]', '', ' ', '[Verse ]'])==False)
joji_ye.show()

+--------------------+
|               value|
+--------------------+
|You left me here,...|
|I played your son...|
|Now I'm reminisci...|
|You said that you...|
|I trusted you lik...|
|You played me lik...|
|So why did I beli...|
|Ooh, ooh-ooh-ooh-...|
|Ooh, ooh-ooh-ooh-...|
|Ooh, oohl (I stil...|
|Oohl (When you to...|
+--------------------+



In [65]:
joji_ye = joji_ye.withColumn("artist", lit('Joji')).select("artist", "value")

In [66]:
joji_ye.toPandas().to_csv("/content/sample_data/Joji_YoungEnergy.csv")

In [67]:
joji_ye = joji_ye.withColumnRenamed("value","formatted_lyrics")
joji_ye = joji_ye.withColumn("label", lit(5.0))
joji_ye.show()

+------+--------------------+-----+
|artist|    formatted_lyrics|label|
+------+--------------------+-----+
|  Joji|You left me here,...|  5.0|
|  Joji|I played your son...|  5.0|
|  Joji|Now I'm reminisci...|  5.0|
|  Joji|You said that you...|  5.0|
|  Joji|I trusted you lik...|  5.0|
|  Joji|You played me lik...|  5.0|
|  Joji|So why did I beli...|  5.0|
|  Joji|Ooh, ooh-ooh-ooh-...|  5.0|
|  Joji|Ooh, ooh-ooh-ooh-...|  5.0|
|  Joji|Ooh, oohl (I stil...|  5.0|
|  Joji|Oohl (When you to...|  5.0|
+------+--------------------+-----+



In [68]:
# Fit and transform the pipeline
joji_cleaner = artist_data_prep_pipeline.fit(joji_ye)
joji_cleaned = joji_cleaner.transform(joji_ye)

In [69]:
joji_test_results = predictor.transform(joji_cleaned)
joji_test_results.show()

+------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|artist|    formatted_lyrics|label|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  Joji|You left me here,...|  5.0|[you, left, me, h...| [left, here,, dark]|(262144,[170414,2...|(262144,[170414,2...|(262144,[170414,2...|[-64.932046046340...|[0.00246844307684...|       6.0|
|  Joji|I played your son...|  5.0|[i, played, your,...|[played, song,, r...|(262144,[71316,10...|(262144,[71316,10...|(262144,[71316,10...|[-83.618309389916...|[0.01519005562984...|       2.0|
|  Joji|Now I'm reminisci...| 

In [70]:
joji_acc = acc_eval.evaluate(joji_test_results)
print("Accuracy of model at predicting artists was: %f" % joji_acc)

Accuracy of model at predicting artists was: 0.166667


In [ ]:
joji_test_results.select('label').describe()

DataFrame[summary: string, label: string]